In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

#loading dataset
(ds_train,ds_test),ds_info = tfds.load(
    'cifar10',
    split = ['train','test'],
    shuffle_files = True,
    as_supervised = True,
    with_info = True
)

In [ ]:
def preprocess(image, label):
    # Normalize pixel values to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    # Resize (optional, here CIFAR is already 32x32)
    image = tf.image.resize(image, [32, 32])
    return image, label

batch_size = 64

train_data = ds_train.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_data = ds_test.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(10, activation='softmax')  # 10 classes → softmax
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # use sparse if labels are integers (not one-hot)
    metrics=['accuracy']
)


In [ ]:
history = model.fit(train_data, validation_data=test_data, epochs=10)

loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Path to your image
img_path = '/content/deer.png'#use your own image path to get classification among 'airplane','car','bird','cat','deer','dog','frog','horse','ship','truck'

# Load image
img = Image.open(img_path)

# Resize to CIFAR-10 input size
img = img.resize((32, 32))

# Convert to numpy array and normalize
img_array = np.array(img) / 255.0

# If image has alpha channel, remove it
if img_array.shape[-1] == 4:
    img_array = img_array[..., :3]

# Add batch dimension
img_array = np.expand_dims(img_array, axis=0)

# Predict
predictions = model.predict(img_array)

# Get predicted class index
predicted_class = np.argmax(predictions[0])

# CIFAR-10 class names
class_names = ['airplane','car','bird','cat','deer','dog','frog','horse','ship','truck']

print(f"Predicted Class ID: {predicted_class}")
print(f"Predicted Class Name: {class_names[predicted_class]}")
